In [1]:
import torch
import sys
from swift.llm import get_model_tokenizer
dirs = ["/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
from custom import CustomModelType, CustomTemplateType
model_type = CustomModelType.openchat_35
model, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)


2024-02-08 05:05:34,659 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-02-08 05:05:34,661 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-02-08 05:05:34,801 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/openchat-3.5-0106",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": fa

In [2]:
import jsonlines
from tqdm.notebook import tqdm

dict_list = []
version = "2"
with jsonlines.open(f"data{version}.jsonl", mode="r") as file_jsonl:
    for item in file_jsonl.iter():
        dict_list.append(item)

lens1 = []
lens2 = []

for item in tqdm(dict_list):
    lens1.append(len(tokenizer(item["query"])["input_ids"]))
    # lens2.append(len(tokenizer.tokenize(item["query"])))



  0%|          | 0/12192 [00:00<?, ?it/s]

In [4]:
# 3705

# [3675, 3674, 3671, 3665, 3662, 3662, 3640, 3638, 3636, 3632, 3616, 3604]


sorted(lens1, reverse=True)[:12]


[3675, 3674, 3671, 3665, 3662, 3662, 3640, 3638, 3636, 3632, 3616, 3604]

修改较长的句子

显卡 24GB

7B模型，长度控制在3600以下 


In [4]:
del_ids = []
for i, item in enumerate(lens1):
    if item == 3675:
        del_ids.append(i)

del_ids


[9298]

In [6]:
# print(len(tokenizer(dict_list[4157]["query"])["input_ids"]))
dict_list[614]["query"]


'According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-11-06\nContent: Former scientific advisor to Pfizer Michael Yeadon claims that the pandemic is fundamentally over in the U.K., and, we now know loads of people had prior immunity to COVID-19\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: 2020-11-28\nTitle: Science Feedback: Ex Pfizer Vice President, Michael Yeadon, made ...\nContent:\n"After drugmaker Pfizer’s former Vice President Michael Yeadon claimed that “the pandemic is fundamentally over in the UK,” health #fact_checker Health Feedback termed his assertions “#false”. KD #Covid_19 https://t.co/2YCTx30Hpq— PIDA STS 2020 (@PidaSts) November 27, 2020 ... \u2066@fordnation\u2069 Dr. Michael Yeadon, VP, Chief Scientist of Pfizer (ret) “no need for va

In [10]:
print(dict_list[9298]["query"])


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-04-06
Content: COVID-19 Linked to Multiple Cardiovascular Presentations

PRIOR KNOWLEDGE:
Information 1:
Publication date: None
Title: CARDIOVASCULAR COMPLICATIONS OF COVID-19 - PMC
Content:
In the initial three years of the pandemic, the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), the etiological agent of COVID-19, has infected more than 600 million individuals causing over 6 million deaths.1 COVID-19 can target multiple systems in the body, including the cardiovascular (CV) system. Multisystemic involvement can be present during the acute phase of the infection and may continue or develop even long-term. This extended presentation of COVID-19 is referred to long COVID, long-haul COVID, and Pos